In [50]:
import pickle
import gc
from datetime import datetime
from os import mkdir
from os.path import join, exists

import numpy as np
import pandas as pd
import scipy as sp

# import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from lightgbm import LGBMClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier

from sklearn import metrics

In [2]:
train_f = pd.read_pickle('../dataset/train/train_all_feature.pkl')
train_f.shape

(879539, 403)

In [3]:
test_f = pd.read_pickle('../dataset/train/test_all_feature.pkl')
test_f.shape

(215287, 403)

In [4]:
model_path = '../dataset/lightgbmmodel.pkl'

In [5]:
luck = 123456

In [6]:
model = LGBMClassifier(nthread=4, silent=False, objective= 'binary', seed=luck)

In [7]:
parameters = {
#     'gamma': [0.05],
    #     'n_estimators': [800],
#     'max_depth': [2, 6, 12],
    'max_depth': [12],
    'num_leaves': range(30, 50, 4), 

#     'learning_rate': [0.15, 0.3],
#         'subsample': [0.9],
#         'colsample_bytree': [0.9],
    #     'reg_alpha': [0, 1, 5],
    #     'reg_lambda': [0, 1, 4],
}

In [8]:
use_columns = [
    x for x in train_f.columns
    if x not in [
        'orderid', 'uid', 'hotelid', 'basicroomid', 'hotel_roomid', 'roomid',
        'orderlabel', 'index'
    ]
]

In [9]:
cv = model_selection.ShuffleSplit(n_splits=1, test_size=0.2, random_state=luck)
clf = model_selection.GridSearchCV(model, parameters, cv=cv, n_jobs=-1, verbose=True)

In [10]:
clf_feture = train_f.sample(100000, random_state=luck)

In [11]:
%time clf.fit(clf_feture[use_columns], clf_feture['orderlabel'])

Fitting 1 folds for each of 5 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   24.0s finished


CPU times: user 17.6 s, sys: 558 ms, total: 18.1 s
Wall time: 30.4 s


GridSearchCV(cv=ShuffleSplit(n_splits=1, random_state=123456, test_size=0.2, train_size=None),
       error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=10, nthread=4,
        num_leaves=31, objective='binary', reg_alpha=0, reg_lambda=0,
        seed=123456, silent=False, subsample=1, subsample_for_bin=50000,
        subsample_freq=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'num_leaves': range(30, 50, 4), 'max_depth': [12]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [12]:
clf.best_score_

0.97465000000000002

In [13]:
clf.best_params_

{'max_depth': 12, 'num_leaves': 30}

In [14]:
# # clf.best_params_['n_estimators'] = 
clf.best_params_['max_depth'] = 6
# clf.best_params_['reg_lambda'] = 0.5
clf.best_params_['subsample'] = 0.8
clf.best_params_['colsample_bytree'] = 0.8

clf.best_params_['colsample_bylevel'] = 0.7

# clf.best_params_['n_estimators'] = 1000
clf.best_params_['learning_rate'] = 0.1
clf.best_params_['num_leaves'] = 56


# clf.best_params_['reg_alpha'] = 1
# clf.best_params_['min_data_in_leaf'] = 3

# clf.best_params_['min_child_weight'] = 1
clf.best_params_['lambda_l2'] = 10
clf.best_params_['tree_method'] = 'exact'
clf.best_params_['metric'] = 'binary_logloss'



In [53]:
params = {
    'boosting_type': 'gbdt',
#     'objective': 'binary',
    'metric': 'auc',
    'min_child_weight': 1.5,
    'num_leaves': 2 ** 5,
    'lambda_l2': 10,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
#     'colsample_bylevel': 0.7,
    'learning_rate': 0.05,
#     'tree_method': 'exact',
}

In [16]:
model = LGBMClassifier(
    nthread=4,
    silent=False,
    objective='binary',
    seed=luck,
#     learning_rate=0.3,
    n_estimators=1300,
    **params)
model

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7, lambda_l2=10,
        learning_rate=0.05, max_bin=255, max_depth=-1,
        metric='binary_logloss', min_child_samples=10,
        min_child_weight=1.5, min_split_gain=0, n_estimators=1300,
        nthread=4, num_leaves=32, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=123456, silent=False, subsample=0.7,
        subsample_for_bin=50000, subsample_freq=1)

In [47]:
# categorical_feature=['order_' + x for x in  ['roomservice_8','roomservice_4','roomservice_3','roomtag_1']]

In [48]:
model.fit(
    train_f[use_columns],
    train_f['orderlabel'],
    eval_set=[(train_f[use_columns],
    train_f['orderlabel']),
             (test_f[use_columns],
    test_f['orderlabel'])],
    eval_metric='auc',
    verbose=True,
    early_stopping_rounds=50,
)

[1]	valid_0's auc: 0.871631	valid_1's auc: 0.870339
Train until valid scores didn't improve in 30 rounds.
[2]	valid_0's auc: 0.894175	valid_1's auc: 0.892031
[3]	valid_0's auc: 0.894845	valid_1's auc: 0.892611
[4]	valid_0's auc: 0.896674	valid_1's auc: 0.894455
[5]	valid_0's auc: 0.896633	valid_1's auc: 0.894425
[6]	valid_0's auc: 0.897397	valid_1's auc: 0.89524
[7]	valid_0's auc: 0.899257	valid_1's auc: 0.897302
[8]	valid_0's auc: 0.899543	valid_1's auc: 0.897307
[9]	valid_0's auc: 0.899884	valid_1's auc: 0.89757
[10]	valid_0's auc: 0.90017	valid_1's auc: 0.897702
[11]	valid_0's auc: 0.900098	valid_1's auc: 0.897706
[12]	valid_0's auc: 0.900306	valid_1's auc: 0.897755
[13]	valid_0's auc: 0.905379	valid_1's auc: 0.902815
[14]	valid_0's auc: 0.905839	valid_1's auc: 0.903505
[15]	valid_0's auc: 0.906274	valid_1's auc: 0.903721
[16]	valid_0's auc: 0.906471	valid_1's auc: 0.903895
[17]	valid_0's auc: 0.906302	valid_1's auc: 0.9036
[18]	valid_0's auc: 0.906309	valid_1's auc: 0.903509
[19]	v

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7, lambda_l2=10,
        learning_rate=0.05, max_bin=255, max_depth=-1,
        metric='binary_logloss', min_child_samples=10,
        min_child_weight=1.5, min_split_gain=0, n_estimators=1300,
        nthread=4, num_leaves=32, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=123456, silent=False, subsample=0.7,
        subsample_for_bin=50000, subsample_freq=1)

In [33]:
model.best_iteration

844

In [34]:
def get_score(model, train_f):
    result = model.predict_proba(train_f[use_columns])[:, 1]
    train_f['prob'] = result
    train_x = train_f[['orderid', 'prob', 'orderlabel']]
    train_x.sort_values('prob', ascending=False, inplace=True)
    finall_result_x = train_x.drop_duplicates(['orderid'])
    train_score = finall_result_x.orderlabel.mean()
    print(train_score)
    return train_score

In [35]:
train_score = get_score(model, train_f)

0.492609425541


/home/zhanglun/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [36]:
test_score = get_score(model, test_f)

0.437305359777


/home/zhanglun/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [37]:
acu = model.score(train_f[use_columns], train_f['orderlabel'])
acu

0.97694815124741485

In [38]:
# y_predict = model.predict_proba(train_features[use_columns])[:, 1]
# fpr, tpr, thresholds  = metrics.roc_curve(train_features['orderlabel'], y_predict, pos_label=1)
# auc_score = metrics.auc(fpr, tpr)
# auc_score

In [39]:
if not exists('models'):
    mkdir('models')
model_importance_path = join('models', datetime.now().strftime('%d-%H%M-importance.txt'))
model_importance_path

'models/06-0817-importance.txt'

In [40]:
importance_df = pd.DataFrame(model.feature_importances_, index=use_columns)

importance_df.sort_values(0, ascending=False, inplace=True)
importance_df.to_csv(model_importance_path)

In [41]:
parms = ','.join(['{}: {}'.format(k, v) for k, v in clf.best_params_.items()])
parms

'metric: binary_logloss,subsample: 0.8,lambda_l2: 10,max_depth: 6,learning_rate: 0.1,colsample_bytree: 0.8,tree_method: exact,num_leaves: 56,colsample_bylevel: 0.7'

In [42]:
parms = ','.join(['{}: {}'.format(k, v) for k, v in params.items()])
parms

'subsample: 0.7,boosting_type: gbdt,metric: binary_logloss,lambda_l2: 10,learning_rate: 0.05,colsample_bytree: 0.7,min_child_weight: 1.5,num_leaves: 32'

In [43]:
print(
    datetime.now().strftime('%d-%H-%M:'),
    'online',
    '??',
    'test_score',
    test_score,
    'train_score',
    train_score,
    'model_train',
    model.best_score['valid_0']['auc'],
    'model_test',
    model.best_score['valid_1']['auc'],
    'accu',
    acu,
    'n',
    model.best_iteration,
    'params',
    parms,
    'shape', train_f.shape,
    end='\n',
    sep='  ')

06-08-17:  online  ??  test_score  0.437305359777  train_score  0.492609425541  model_train  0.959623963322  model_test  0.935383612908  accu  0.976948151247  n  844  params  subsample: 0.7,boosting_type: gbdt,metric: binary_logloss,lambda_l2: 10,learning_rate: 0.05,colsample_bytree: 0.7,min_child_weight: 1.5,num_leaves: 32  shape  (879539, 404)


In [44]:
print(
    datetime.now().strftime('%d-%H-%M:'),
    'online',
    '??',
    'test_score',
    test_score,
    'train_score',
    train_score,
    'model_train',
    model.best_score['valid_0']['auc'],
    'model_test',
    model.best_score['valid_1']['auc'],
    'accu',
    acu,
    'n',
    model.best_iteration,
    'params',
    parms,
    'shape', train_f.shape,
    end='\n',
    sep='  ', file=open('lgb_result.txt', 'a+'))

In [49]:
del train_f, test_f

In [54]:
gc.collect()

174

In [55]:
train_f = pd.read_pickle('../dataset/train/all_feature.pkl')
train_f.shape

(1094826, 403)

In [56]:
n_est = model.best_iteration + 200 if model.best_iteration > 0 else 1500

In [57]:
model = LGBMClassifier(
    nthread=4,
    silent=False,
    objective='binary',
    seed=luck,
#     learning_rate=0.3,
    n_estimators=n_est,
    **params)
model

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7, lambda_l2=10,
        learning_rate=0.05, max_bin=255, max_depth=-1, metric='auc',
        min_child_samples=10, min_child_weight=1.5, min_split_gain=0,
        n_estimators=993, nthread=4, num_leaves=32, objective='binary',
        reg_alpha=0, reg_lambda=0, seed=123456, silent=False,
        subsample=0.7, subsample_for_bin=50000, subsample_freq=1)

In [47]:
# categorical_feature=['order_' + x for x in  ['roomservice_8','roomservice_4','roomservice_3','roomtag_1']]

In [58]:
model.fit(
    train_f[use_columns],
    train_f['orderlabel'],
    eval_set=[(train_f[use_columns],
    train_f['orderlabel'])],
    eval_metric='auc',
    verbose=True,
)

[1]	valid_0's auc: 0.8724
[2]	valid_0's auc: 0.893036
[3]	valid_0's auc: 0.897135
[4]	valid_0's auc: 0.897226
[5]	valid_0's auc: 0.89691
[6]	valid_0's auc: 0.897969
[7]	valid_0's auc: 0.899559
[8]	valid_0's auc: 0.904696
[9]	valid_0's auc: 0.904979
[10]	valid_0's auc: 0.905353
[11]	valid_0's auc: 0.905337
[12]	valid_0's auc: 0.905241
[13]	valid_0's auc: 0.905095
[14]	valid_0's auc: 0.906015
[15]	valid_0's auc: 0.906088
[16]	valid_0's auc: 0.906224
[17]	valid_0's auc: 0.906331
[18]	valid_0's auc: 0.907928
[19]	valid_0's auc: 0.907883
[20]	valid_0's auc: 0.909982
[21]	valid_0's auc: 0.910449
[22]	valid_0's auc: 0.910703
[23]	valid_0's auc: 0.910529
[24]	valid_0's auc: 0.911062
[25]	valid_0's auc: 0.911264
[26]	valid_0's auc: 0.91198
[27]	valid_0's auc: 0.91238
[28]	valid_0's auc: 0.912936
[29]	valid_0's auc: 0.913119
[30]	valid_0's auc: 0.91357
[31]	valid_0's auc: 0.913914
[32]	valid_0's auc: 0.914464
[33]	valid_0's auc: 0.914666
[34]	valid_0's auc: 0.914715
[35]	valid_0's auc: 0.915353


LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7, lambda_l2=10,
        learning_rate=0.05, max_bin=255, max_depth=-1, metric='auc',
        min_child_samples=10, min_child_weight=1.5, min_split_gain=0,
        n_estimators=993, nthread=4, num_leaves=32, objective='binary',
        reg_alpha=0, reg_lambda=0, seed=123456, silent=False,
        subsample=0.7, subsample_for_bin=50000, subsample_freq=1)

In [59]:
pickle.dump(model, open(model_path, 'wb'))

In [60]:
# clf1 =  LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=42, max_depth=6)
# clf2 = LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=32, max_depth=5)
# clf3 = LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=27, max_depth=3)
# lr = LogisticRegression()
# sclf = StackingCVClassifier(classifiers=[clf1, clf2, clf3], 
#                           meta_classifier=lr)

# model = sclf